## 1. Cargar los datos

#### 1.1. Importar librerias y módulos

In [1]:
import pandas as pd
import numpy as np
from glob import glob
import os
import funciones as fun
import variables_nombres as vn
from warnings import simplefilter

In [2]:
#print(dir(fun))
simplefilter( action = "ignore", category = pd.errors.PerformanceWarning )
#pd.set_option('display.max_rows', 10)
#pd.set_option('display.max_columns', 10)

#### 1.2. Importar bases 

Se importan las cuatro bases de manera conjunta, y se agrupan en la lista "bases". Para accerder a cada una de las bases se llama a la lista de bases + su respectivo índice. Ejemplo: bases[0]

In [3]:
pwd

'D:\\Users\\u_sociales\\Documents\\Josue\\Corruption_Paper\\code\\preprocess_data'

In [4]:
path = r'..\..\output\data_build\*.dta'

In [5]:
%%time

bases = fun.importar_bases( path )
for i, base in enumerate( bases ):
    print( i )

0
1
2
3
Wall time: 2min 24s


In [6]:
bases[ 3 ]

,ubigeo,ccdd,ccpp,ccdi,Departamento,Provincia,Distrito,catmuni,year,VFI,...,monto_objeto_promedio,tipo_control,corrup_intensa,corrup_amplia,monto_,monto,per_corrup1,per_corrup2,monto_corrup1,monto_corrup2
0,010102,01,01,02,AMAZONAS,CHACHAPOYAS,ASUNCION,2,2017.0,1,...,7662770.0,2,1,1,.,7.662770e+06,5.0,0.0,7.662770e+06,7.662770e+06
1,010108,01,01,08,AMAZONAS,CHACHAPOYAS,HUANCAS,2,2017.0,1,...,172923.0,2,1,1,.,1.729230e+05,3.0,0.0,1.729230e+05,1.729230e+05
2,010119,01,01,19,AMAZONAS,CHACHAPOYAS,SAN ISIDRO DE MAINO,2,2017.0,1,...,.,1,1,1,.,5.593400e+06,4.0,0.0,5.593400e+06,5.593400e+06
3,010202,,,,,,,.,2012.0,.,...,.,1,1,1,.,6.749438e+05,7.0,0.0,6.749438e+05,6.749438e+05
4,010202,,,,,,,.,2013.0,.,...,.,1,1,1,.,6.749438e+05,7.0,0.0,6.749438e+05,6.749438e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,250107,25,01,07,UCAYALI,CORONEL PORTILLO,MANANTAY,1,2018.0,1,...,145801.140625,2,0,0,.,1.458011e+05,0.0,0.0,0.000000e+00,0.000000e+00
1965,250107,25,01,07,UCAYALI,CORONEL PORTILLO,MANANTAY,1,2018.0,1,...,.,1,0,0,.,1.037036e+05,0.0,0.0,0.000000e+00,0.000000e+00
1966,250107,25,01,07,UCAYALI,CORONEL PORTILLO,MANANTAY,1,2018.0,1,...,0.0,2,1,1,.,0.000000e+00,1.0,0.0,0.000000e+00,0.000000e+00
1967,250203,25,02,03,UCAYALI,ATALAYA,TAHUANIA,2,2019.0,1,...,1000000.0,2,1,1,.,1.000000e+06,5.0,0.0,1.000000e+06,1.000000e+06


In [7]:
# Filtrar los años

for i, base in enumerate( bases ):
    bases[ i ] = bases[ i ][ bases[ i ][ 'year' ] >= 2016 ]
bases[ 1 ][ 'year' ].value_counts( normalize = True )

2018.0    0.280072
2016.0    0.254937
2019.0    0.224417
2017.0    0.175943
2020.0    0.064632
Name: year, dtype: float64

#### 1.3. Filtrar solo aquellas variables que hicieron match

Se filtra las variables que previamente, en el preprocesamiento en Stata, hicieron match en la realización de merge.

In [ ]:
# for i, base in enumerate( bases ):
    # bases[i] = bases[i].loc[ bases[i]['_merge'] == 3 ]

#### 1.4. Última limpieza a las bases

Últimas modificaciones a las bases de datos

In [8]:
%%time

for i, base in enumerate( bases ):
    data_cols = [ col for col in bases[i].columns if col not in vn.no_predictoras_variables and col not in vn.string_variables ]
    encode_variables = { "sexo":{ "Mujer":1, "Hombre":2 }}
    bases[i] = bases[i][data_cols]
    bases[i] = bases[i].replace( encode_variables )
    bases[i] = bases[i].astype( str ).replace( '', np.nan, regex = True )
    bases[i] = bases[i].astype( str ).replace( '\.+$', np.nan, regex = True ).astype( float )

Wall time: 37.3 s


#### 1.5. Duplicar las bases de datos finales

In [13]:
dfs = [ pd.DataFrame() for x in range( 4 ) ]
for base in bases:
    dfs = bases.copy()

#### 1.6. Asignar nombres a las bases

In [21]:
dfs[0].name = 'base0'
dfs[1].name = 'base1'
dfs[2].name = 'base2'
dfs[3].name = 'base3'

#### 1.7. Determinar el número de variables por cada fuente

In [15]:
fun.contar_variables( dfs )

base0: Variables de Renamu: 610; Variables de SIAF: 8764
base1: Variables de Renamu: 610; Variables de SIAF: 8764
base2: Variables de Renamu: 610; Variables de SIAF: 8764
base3: Variables de Renamu: 610; Variables de SIAF: 8764


#### 1.8. Determinar las dimensiones de cada base

In [16]:
fun.determinar_dimensiones( dfs )

base0 (419, 9380)
base1 (557, 9380)
base2 (854, 9380)
base3 (1400, 9383)


## 2. Primera forma: imputación con 0

#### 2.1. Imputación

Se imputa los valores perdidos de las variables de SIAF con 0. No se generan variables dummy de control.

In [17]:
fun.imputar_i( dfs, vn.siaf_variables, 0, dummy = False )

[     VFI  facebook  web  propie_muni  locales_propios  mue_pro  mue_pro_1  \
 0    1.0       0.0  1.0          1.0              NaN      1.0        1.0   
 1    1.0       0.0  0.0          1.0              NaN      1.0        1.0   
 2    1.0       0.0  0.0          1.0              NaN      1.0        1.0   
 4    1.0       1.0  0.0          0.0              NaN      1.0        1.0   
 6    1.0       0.0  0.0          1.0              NaN      1.0        0.0   
 ..   ...       ...  ...          ...              ...      ...        ...   
 647  1.0       1.0  1.0          1.0              NaN      1.0        1.0   
 648  1.0       1.0  1.0          1.0              3.0      1.0        1.0   
 649  1.0       0.0  0.0          1.0              1.0      1.0        1.0   
 650  1.0       1.0  1.0          0.0              NaN      1.0        1.0   
 651  1.0       1.0  1.0          1.0              NaN      1.0        1.0   
 
      mue_pro_2  mue_pro_3  mue_pro_4  ...  _tdvgtotfun_f5viv 

Generar tablas descriptivas de los cuatro casos

In [18]:
export_path = r'..\..\extra\descriptive_tables'
file_name = "tabla_descriptiva_filtrada_ac"

In [19]:
fun.generar_tablas_descriptivas( dfs, file_name, export_path ) 

#### 2.2. Filtrar por valores perdidos

Se descartan las variables con un porcentaje de valores perdidos mayor o igual al umbral 0.5

In [20]:
fun.filtro_missings( dfs, 0.1 )

[     VFI  facebook  web  propie_muni  mue_pro  mue_pro_1  mue_pro_2  \
 0    1.0       0.0  1.0          1.0      1.0        1.0        0.0   
 1    1.0       0.0  0.0          1.0      1.0        1.0        1.0   
 2    1.0       0.0  0.0          1.0      1.0        1.0        3.0   
 4    1.0       1.0  0.0          0.0      1.0        1.0        1.0   
 6    1.0       0.0  0.0          1.0      1.0        0.0        NaN   
 ..   ...       ...  ...          ...      ...        ...        ...   
 647  1.0       1.0  1.0          1.0      1.0        1.0        4.0   
 648  1.0       1.0  1.0          1.0      1.0        1.0        3.0   
 649  1.0       0.0  0.0          1.0      1.0        1.0        1.0   
 650  1.0       1.0  1.0          0.0      1.0        1.0        3.0   
 651  1.0       1.0  1.0          1.0      1.0        1.0        1.0   
 
      mue_pro_3  mue_pro_4  mue_pro_7  ...  _tdvgtotfun_f5viv  \
 0          1.0        2.0        2.0  ...        5000.000000   
 1  

Se verifica el numero de variables por fuente de cada base luego del paso 2.2.

In [22]:
fun.contar_variables( dfs )

base0: Variables de Renamu: 275; Variables de SIAF: 8764
base1: Variables de Renamu: 278; Variables de SIAF: 8764
base2: Variables de Renamu: 276; Variables de SIAF: 8764
base3: Variables de Renamu: 279; Variables de SIAF: 8764


Se verifica la dimensionalidad de las bases de datos

In [23]:
fun.determinar_dimensiones( dfs )

base0 (419, 9045)
base1 (557, 9048)
base2 (854, 9046)
base3 (1400, 9049)


Se verifica que las variables pertenecientes a SIAF ya no tengan valores perdidos

In [24]:
df0_siaf = fun.listar_variables( dfs[0], siaf = True )
df1_siaf = fun.listar_variables( dfs[1], siaf = True )
df2_siaf = fun.listar_variables( dfs[2], siaf = True )
df3_siaf = fun.listar_variables( dfs[3], siaf = True )

In [25]:
print( dfs[0][df0_siaf].isnull().sum().sum(),
       dfs[1][df1_siaf].isnull().sum().sum(),
       dfs[2][df2_siaf].isnull().sum().sum(),
       dfs[3][df3_siaf].isnull().sum().sum(), sep = "\n" )

0
0
0
0


#### 2.3. Imputación con media y moda para variables de Renamu

In [26]:
[ fun.imputar_ii( df, vn.renamu_numvars, num = True, dummy = False ) for df in dfs ]

C:\Users\dell\Documents\QLAB\Corruption_Paper\code\preprocess_data\funciones.py:341: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[var] = dataframe[var].fillna( media )


[     VFI  facebook  web  propie_muni  mue_pro  mue_pro_1  mue_pro_2  \
 0    1.0       0.0  1.0          1.0      1.0        1.0   0.000000   
 1    1.0       0.0  0.0          1.0      1.0        1.0   1.000000   
 2    1.0       0.0  0.0          1.0      1.0        1.0   3.000000   
 4    1.0       1.0  0.0          0.0      1.0        1.0   1.000000   
 6    1.0       0.0  0.0          1.0      1.0        0.0  15.583548   
 ..   ...       ...  ...          ...      ...        ...        ...   
 647  1.0       1.0  1.0          1.0      1.0        1.0   4.000000   
 648  1.0       1.0  1.0          1.0      1.0        1.0   3.000000   
 649  1.0       0.0  0.0          1.0      1.0        1.0   1.000000   
 650  1.0       1.0  1.0          0.0      1.0        1.0   3.000000   
 651  1.0       1.0  1.0          1.0      1.0        1.0   1.000000   
 
      mue_pro_3  mue_pro_4  mue_pro_7  ...  _tdvgtotfun_f5viv  \
 0     1.000000        2.0        2.0  ...        5000.000000   
 1  

In [27]:
[ fun.imputar_ii( df, vn.renamu_catvars, num = False, dummy = False ) for df in dfs ]

C:\Users\dell\Documents\QLAB\Corruption_Paper\code\preprocess_data\funciones.py:348: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[var] = dataframe[var].fillna( moda )
C:\Users\dell\Documents\QLAB\Corruption_Paper\code\preprocess_data\funciones.py:348: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[var] = dataframe[var].fillna( moda )
C:\Users\dell\Documents\QLAB\Corruption_Paper\code\preprocess_data\funciones.py:348: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sli

[     VFI  facebook  web  propie_muni  mue_pro  mue_pro_1  mue_pro_2  \
 0    1.0       0.0  1.0          1.0      1.0        1.0   0.000000   
 1    1.0       0.0  0.0          1.0      1.0        1.0   1.000000   
 2    1.0       0.0  0.0          1.0      1.0        1.0   3.000000   
 4    1.0       1.0  0.0          0.0      1.0        1.0   1.000000   
 6    1.0       0.0  0.0          1.0      1.0        0.0  15.583548   
 ..   ...       ...  ...          ...      ...        ...        ...   
 647  1.0       1.0  1.0          1.0      1.0        1.0   4.000000   
 648  1.0       1.0  1.0          1.0      1.0        1.0   3.000000   
 649  1.0       0.0  0.0          1.0      1.0        1.0   1.000000   
 650  1.0       1.0  1.0          0.0      1.0        1.0   3.000000   
 651  1.0       1.0  1.0          1.0      1.0        1.0   1.000000   
 
      mue_pro_3  mue_pro_4  mue_pro_7  ...  _tdvgtotfun_f5viv  \
 0     1.000000        2.0        2.0  ...        5000.000000   
 1  

In [28]:
df0_renamu = fun.listar_variables( dfs[0], siaf = False )
df1_renamu = fun.listar_variables( dfs[1], siaf = False )
df2_renamu = fun.listar_variables( dfs[2], siaf = False )
df3_renamu = fun.listar_variables( dfs[3], siaf = False )

In [29]:
print( dfs[0].isnull().sum().sum(),
       dfs[1].isnull().sum().sum(),
       dfs[2].isnull().sum().sum(),
       dfs[3].isnull().sum().sum(), sep = "\n" )

0
0
0
0


#### 2.3. Filtro por variabilidad

Se descartan aquellas variables constantes, es decir, con una variabilidad de 0

In [30]:
fun.filtro_variabilidad( dfs, 0 )

C:\Users\dell\Documents\QLAB\Corruption_Paper\code\preprocess_data\funciones.py:239: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datasets[i].drop( columnas, axis = 1, inplace = True )
C:\Users\dell\Documents\QLAB\Corruption_Paper\code\preprocess_data\funciones.py:239: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datasets[i].drop( columnas, axis = 1, inplace = True )
C:\Users\dell\Documents\QLAB\Corruption_Paper\code\preprocess_data\funciones.py:239: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

[     facebook  web  propie_muni  mue_pro_1  mue_pro_2  mue_pro_3  mue_pro_4  \
 0         0.0  1.0          1.0        1.0   0.000000   1.000000        2.0   
 1         0.0  0.0          1.0        1.0   1.000000   0.000000        1.0   
 2         0.0  0.0          1.0        1.0   3.000000   0.000000        2.0   
 4         1.0  0.0          0.0        1.0   1.000000   1.000000        1.0   
 6         0.0  0.0          1.0        0.0  15.583548   4.277635        1.0   
 ..        ...  ...          ...        ...        ...        ...        ...   
 647       1.0  1.0          1.0        1.0   4.000000   4.000000        1.0   
 648       1.0  1.0          1.0        1.0   3.000000   3.000000        1.0   
 649       0.0  0.0          1.0        1.0   1.000000   1.000000        1.0   
 650       1.0  1.0          0.0        1.0   3.000000   0.000000        1.0   
 651       1.0  1.0          1.0        1.0   1.000000   0.000000        0.0   
 
      mue_pro_7  mue_pro_10  mue_pro_1

Se verifica la dimensionalidad de las bases tras el paso 2.3.

In [31]:
fun.determinar_dimensiones( dfs )

base0 (419, 6813)
base1 (557, 6816)
base2 (854, 6946)
base3 (1400, 6949)


#### 2.5. Imputar outliers

Se imputa los valores del percentil 1% superior con los valores del percentil 99% para cada una de las variables de SIAF. En este caso, la imputación se realiza sobre las tres bases resultantes de la base0.

In [32]:
fun.imputar_outliers( dfs, vn.siaf_variables, 0.01 )

C:\Users\dell\Documents\QLAB\Corruption_Paper\code\preprocess_data\funciones.py:430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe.loc[ dataframe[var] > perc, var ] = perc


[     facebook  web  propie_muni  mue_pro_1  mue_pro_2  mue_pro_3  mue_pro_4  \
 0         0.0  1.0          1.0        1.0   0.000000   1.000000        2.0   
 1         0.0  0.0          1.0        1.0   1.000000   0.000000        1.0   
 2         0.0  0.0          1.0        1.0   3.000000   0.000000        2.0   
 4         1.0  0.0          0.0        1.0   1.000000   1.000000        1.0   
 6         0.0  0.0          1.0        0.0  15.583548   4.277635        1.0   
 ..        ...  ...          ...        ...        ...        ...        ...   
 647       1.0  1.0          1.0        1.0   4.000000   4.000000        1.0   
 648       1.0  1.0          1.0        1.0   3.000000   3.000000        1.0   
 649       0.0  0.0          1.0        1.0   1.000000   1.000000        1.0   
 650       1.0  1.0          0.0        1.0   3.000000   0.000000        1.0   
 651       1.0  1.0          1.0        1.0   1.000000   0.000000        0.0   
 
      mue_pro_7  mue_pro_10  mue_pro_1

#### 2.6. Transformaciones logarítmicas

Primero, se identifica las variables negativas y se les divide entre un millón. Segundo, se realiza una transformación logarítmica a todas las variables pertenecientes a SIAF que no son negativas. Estas transformaciones se aplican sobre las tres bases de datos resultantes de la base0.

In [33]:
fun.dividir_variables_negativas( dfs )

C:\Users\dell\Documents\QLAB\Corruption_Paper\code\preprocess_data\funciones.py:439: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[ var ] = dataset[ var ] / 1_000_000


[     facebook  web  propie_muni  mue_pro_1  mue_pro_2  mue_pro_3  mue_pro_4  \
 0         0.0  1.0          1.0        1.0   0.000000   1.000000        2.0   
 1         0.0  0.0          1.0        1.0   1.000000   0.000000        1.0   
 2         0.0  0.0          1.0        1.0   3.000000   0.000000        2.0   
 4         1.0  0.0          0.0        1.0   1.000000   1.000000        1.0   
 6         0.0  0.0          1.0        0.0  15.583548   4.277635        1.0   
 ..        ...  ...          ...        ...        ...        ...        ...   
 647       1.0  1.0          1.0        1.0   4.000000   4.000000        1.0   
 648       1.0  1.0          1.0        1.0   3.000000   3.000000        1.0   
 649       0.0  0.0          1.0        1.0   1.000000   1.000000        1.0   
 650       1.0  1.0          0.0        1.0   3.000000   0.000000        1.0   
 651       1.0  1.0          1.0        1.0   1.000000   0.000000        0.0   
 
      mue_pro_7  mue_pro_10  mue_pro_1

In [34]:
fun.transformacion_log( dfs )

C:\Users\dell\Documents\QLAB\Corruption_Paper\code\preprocess_data\funciones.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datasets[ i ][ var ] = np.log(datasets[ i ][ var ] + 1)


[     facebook  web  propie_muni  mue_pro_1  mue_pro_2  mue_pro_3  mue_pro_4  \
 0         0.0  1.0          1.0        1.0   0.000000   1.000000        2.0   
 1         0.0  0.0          1.0        1.0   1.000000   0.000000        1.0   
 2         0.0  0.0          1.0        1.0   3.000000   0.000000        2.0   
 4         1.0  0.0          0.0        1.0   1.000000   1.000000        1.0   
 6         0.0  0.0          1.0        0.0  15.583548   4.277635        1.0   
 ..        ...  ...          ...        ...        ...        ...        ...   
 647       1.0  1.0          1.0        1.0   4.000000   4.000000        1.0   
 648       1.0  1.0          1.0        1.0   3.000000   3.000000        1.0   
 649       0.0  0.0          1.0        1.0   1.000000   1.000000        1.0   
 650       1.0  1.0          0.0        1.0   3.000000   0.000000        1.0   
 651       1.0  1.0          1.0        1.0   1.000000   0.000000        0.0   
 
      mue_pro_7  mue_pro_10  mue_pro_1

In [35]:
print( dfs[0].isnull().sum().sum(),
       dfs[1].isnull().sum().sum(),
       dfs[2].isnull().sum().sum(),
       dfs[3].isnull().sum().sum(), sep = "\n" )

0
0
0
0


#### 2.7. Exportar las bases de datos

In [36]:
len( dfs )

4

In [55]:
# Total actual

# dfs[0].to_csv( r'..\..\output\data_preprocess\base0_ac.csv', index = False )
# dfs[1].to_csv( r'..\..\output\data_preprocess\base1_ac.csv', index = False )
# dfs[2].to_csv( r'..\..\output\data_preprocess\base2_ac.csv', index = False )
# dfs[3].to_csv( r'..\..\output\data_preprocess\base3_ac.csv', index = False )

In [37]:
dfs[0].to_csv( r'..\..\output\data_preprocess\base0_ac_last.csv', index = False )
dfs[1].to_csv( r'..\..\output\data_preprocess\base1_ac_last.csv', index = False )
dfs[2].to_csv( r'..\..\output\data_preprocess\base2_ac_last.csv', index = False )
dfs[3].to_csv( r'..\..\output\data_preprocess\base3_ac_last.csv', index = False )